In [1]:
"""
Data wrangling for the Yahoo! Front Page Today Module User Click Log Dataset, version 1.0.

Inspired by:
Unbiased Offline Evaluation of Contextual-bandit-based News Article Recommendation Algorithms 
[https://arxiv.org/pdf/1003.5956.pdf]

Documentation is per reST format used in Sphinx.

Dataset: https://webscope.sandbox.yahoo.com/catalog.php?datatype=r&did=49
Author: jtcho (jonathan.t.cho@gmail.com)

Many thanks to Yahoo! Research for allowing me to use their dataset.
"""

import pandas as pd
import numpy as np
import sqlite3
import time
import os

dump_dir = 'R6/'
data_dirs = ['clicks_1/']
engine = sqlite3.connect('yahoo')

In [2]:
# Database cleanup.

c = engine.cursor()
c.execute('DROP TABLE articles')
engine.commit()

In [3]:
def extract_article_info(path, item_limit=sys.maxsize):
    """ 
    Given an R6A dataset file, extracts all of the common article vectors
    and compiles them in a single dataframe.
    Note that each article has a constant vector associated with it.
    
    :param path:       the file path for the dataset
    :param item_limit: limits the number of items to parse
    :returns: Pandas dataframe containing article vectors indexed by id
    """
    t0 = time.time()
    num_iters = 0
    _articles_df = pd.DataFrame(columns=['2', '3', '4', '5', '6', '1'])
    with open(path) as f:
        for line in f:
            num_iters += 1 
            if num_iters > item_limit:
                break
            parts = line.strip().split('|')
            for i in range(2, len(parts)):
                # Extract article vector information.
                article_info = parts[i].split()
                article_id = article_info[0]
                if article_id in _articles_df.index:
                    continue
                article_info_parts = list(map(lambda x : x.split(':')[1], article_info[1:]))
                article_info = dict(zip(_articles_df.columns, article_info_parts))
                # I append to an existing DF for quick de-duplication. Also
                # empirically, I observed that there is a small number of unique
                # articles for any dataset, so the overhead of doing this is minimized.
                _articles_df.loc[article_id] = pd.Series(article_info)

    t1 = time.time()
    print('Finished processing {0} items in {1} seconds.'.format(num_iters-1, t1 - t0))
    return _articles_df

In [7]:
def process_click_file(path, item_limit=sys.maxsize):
    """
    Given an R6A dataset file, parses all of the view event logs and 
    compiles them in a single dataframe.
    
    A single view event consists of a unix timestamp, a 6-dimensional vector of
    features describing the user, a set of 20 articles in the article pool
    (the 20 arms of the multi-arm bandit), the id of the article displayed, and
    a boolean marking whether the article was clicked.
    """
    t0 = time.time()
    num_iters = 0
    views_cols = ['time', 'user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6', 
                  'article_pool', 'displayed', 'clicked']
    views = []
    with open(path) as f:
        for line in f:
            num_iters += 1
            if num_iters > item_limit:
                break
            parts = line.strip().split('|')
            unix_timestamp, disp_article_id, clicked = parts[0].split()
            user_info = list(map(lambda x : x.split(':')[1], parts[1].split()[1:]))
            user_info = dict(zip(views_cols[1:7], user_info))
            user_info['time'] = unix_timestamp
            user_info['displayed'] = disp_article_id
            user_info['clicked'] = clicked
    
            # Extract article vector information.
            article_ids = [parts[i].split()[0] for i in range(2, len(parts))]
            user_info['article_pool'] = article_ids
            # In this case, we construct the DF at the end because we're creating a new row
            # for *every* item... over ~4 million items that becomes very expensive!
            views.append(user_info)

    t1 = time.time()
    print('{0}: Finished processing {1} items in {2} seconds.'.format(path, num_iters-1, t1 - t0))
    return pd.DataFrame(views, columns=views_cols)

In [4]:
# Run to populate the articles table.
articles_df = extract_article_info(dump_dir + 'clicks_1.txt', sys.maxsize).apply(pd.to_numeric)
articles_df.to_sql('articles', engine, if_exists='replace')

Finished processing 4681991 items in 150.5566005706787 seconds.


In [9]:
for fname in os.listdir('clicks_1'):
    if fname != '.DS_Store':
        result = process_click_file('clicks_1/'+fname)
        result['article_pool'] = result['article_pool'].astype(str)
        result.to_sql('clicks', engine, if_exists='append')

clicks_1/xaa: Finished processing 99999 items in 3.1617259979248047 seconds.
clicks_1/xab: Finished processing 99999 items in 3.2025344371795654 seconds.
clicks_1/xac: Finished processing 99999 items in 3.3164455890655518 seconds.
clicks_1/xad: Finished processing 99999 items in 3.380336046218872 seconds.
clicks_1/xae: Finished processing 99999 items in 3.0821828842163086 seconds.
clicks_1/xaf: Finished processing 99999 items in 3.1906492710113525 seconds.
clicks_1/xag: Finished processing 99999 items in 3.3087258338928223 seconds.
clicks_1/xah: Finished processing 99999 items in 3.2571945190429688 seconds.
clicks_1/xai: Finished processing 99999 items in 3.278446674346924 seconds.
clicks_1/xaj: Finished processing 99999 items in 3.2920501232147217 seconds.
clicks_1/xak: Finished processing 99999 items in 3.431187629699707 seconds.
clicks_1/xal: Finished processing 99999 items in 3.40493106842041 seconds.
clicks_1/xam: Finished processing 99999 items in 3.1150004863739014 seconds.
clic

In [18]:
#pd.read_sql_query('select * from articles',con=engine).set_index('index')

In [11]:
pd.read_sql_query('select count(*) from clicks', con=engine)

,count(*)
0,4681992
